In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#default_exp learner

# Learner

Leaner to train the model

In [ ]:
#export
from PrimeCNNv3.imports import *
from torch.optim.lr_scheduler import _LRScheduler
from PrimeCNNv3.utils.vizualize import plot_lr
import copy

In [ ]:
#export
class Learner:
    '''Learner for the Model'''
    def __init__(self, model, dls, metric, cbs, loss_func, opt_func = None, lr_schedular_func = None):
        self.model = model
        self.dls = dls
        self.device = torch.device(dls.device)
        self.metric = metric
        self.cbs = cbs
        self.loss_func = loss_func
        self.opt_func = opt_func
        self.lr_schedular_func = lr_schedular_func
        
        self.opt = None
        self.lr_schedular = None
        
        #get the number of layer groups in the model
        self.layer_groups = len([*self.model.children()])
        
        for cb in self.cbs:
            cb.learner = self
        
    
    def lr_finder(self, final_lr = 10, start_lr = 1e-08,num_iter = 100, wd = 0, beta = 0.98, suggestion = True):
        
        #save model, optimizer and lr_schdeular
        saved_param = {
            'model' : self.model.state_dict(),
            'optimizer' : self.opt.state_dict() if self.opt is not None else None,
            'lr_schedular' : self.lr_schedular if self.lr_schedular is not None else None
        }
        
        torch.save(saved_param, 'tmp.pth')
        
        
        store_cbs = self.cbs
        self.cbs = []
        avg_loss = 0.
        best_loss = 0.
        
        lrs = []
        lr_loss = []
        self.num_iter = num_iter
        self.model.to(self.device)
        self.training = True
    
        self._init_optimizer(epochs = None, lr= start_lr, wd = wd)
        self.lr_schedular = ExponentialLR(self.opt,final_lr= final_lr, num_iter= self.num_iter)
        
        for self.num,batch in  enumerate(progress_bar(self.dls.train, leave = False)):
            xb, yb = batch
            #xb,yb = iter(self.dls.train).next()
            self.xb = xb.to(self.device, non_blocking = True)
            self.yb = yb.to(self.device, non_blocking = True)
            
            #do one batch
            self.do_batch()
            
            #take moving average
            avg_loss = avg_loss * beta + (1-beta) * self.running_loss
            smooth_loss = avg_loss / (1 - beta**(self.num + 1))
            
            if self.num + 1 > 1 and smooth_loss > 4 * best_loss:
                print('Loss exploding.. stopping training')
                break
            
            if smooth_loss < best_loss or self.num + 1 == 1:
                best_loss = smooth_loss
                
            if self.num == self.num_iter:
                break
            #refactor later
            lr_loss.append(smooth_loss)
            lrs.append(self.lr_schedular.get_last_lr())
        
        if suggestion:
            lr_s, losses = torch.tensor(lrs[num_iter // 10 : -5]), torch.tensor(lr_loss[num_iter // 10 :-5])
            if not (len(losses) == 0):
                lr_min = lr_s[losses.argmin()].item()
                grad = (losses[1:] - losses[:-1]) / (lr_s[1:].log() - lr_s[:-1].log())
                
                lr_steep = lr_s[grad.argmin()].item()
                
                print(f'Suggested LR : \nmin_lr: {lr_min / 10} \nlr_steep: {lr_steep}' )
            else:
                print(f'Length of Loss is zero, NO suggestion')
        self.xb, self.yb = (None,),(None,)
        self.preds = None
        self.loss = None
        
        plot_lr(lrs, lr_loss)
        
        self.cbs = store_cbs
        saved_param = torch.load('tmp.pth')
        self.model.load_state_dict(saved_param['model'])
        self.xb, self.yb = None, None
        #if saved_param['optimizer'] is None:
            #self.opt = None
        #else:
            #self.opt.load_state_dict(saved_param['optimizer'])
        
        if saved_param['lr_schedular'] is None:
            self.lr_schedular = None
        else:
            self.lr_schedular = saved_param['lr_schedular']
            
        
    
    def do_batch(self):
        self('before_batch')
        self.batch_size = len(self.yb)
        self.preds = self.model(self.xb)
        self.loss = self.loss_func(self.preds, self.yb)
        self.running_loss = self.loss.item()
        
        if self.training:
            self('before_step')
            self.opt.zero_grad()
            self.loss.backward()
            self.opt.step()
            self('after_step')
            
            if self.lr_schedular is not None:
                self.lr_schedular.step()
                
        self('after_batch')
      
    
    def do_valid_epoch(self):
        self('before_valid_epoch')
    
        dl = self.dls.valid
        
        for self.num, self.batch in enumerate(progress_bar(dl, parent= self.mb)):
            with torch.no_grad():
                self.do_batch()
                
        self('after_valid_epoch')
   
    def do_train_epoch(self):
        self('before_train_epoch')
        
        dl = self.dls.train
        for self.num, self.batch in enumerate(progress_bar(dl, parent= self.mb)):
            self.do_batch()
            
        self('after_train_epoch')
    
    
    def fit(self, epochs, lr, wd, **kwargs):
        
        
        self._init_optimizer(epochs, lr, wd, **kwargs)
        
        #if self.lr_schedular_func is not None:
            #only for one cycle (Note: can be set at before_fit callback)
           # self.lr_schedular = torch.optim.lr_scheduler.OneCycleLR(self.opt, max_lr = lr, epochs = epochs,
                                                                #steps_per_epoch = len(self.dls.train))
        self('before_fit')
        try:
          
            for self.epoch in self.mb:
                self('before_epoch')
                self.do_train_epoch()
                self.do_valid_epoch()
                self('after_epoch')

        except FitCancelException: pass
        
        self('after_fit')
        
    def fine_tune(self, epochs, max_lr, wd, freeze_epoch = 3, initial_freeze = 7, div_factor = 15., pct_start= 0.25,final_div_factor =1e4,lr_div = 10, **kwargs):
        '''
            Initially freezes layers last layers i.e fully connected layer based on initial freeze value.
            Trains those layer with one_cycle_fit policy for 5 epochs, later, unfreezes all the layers, 
            train for given number of epochs
            
            initial freeze as per resnet
        '''
        self.freeze_to(freeze_to_till = initial_freeze,show_log = True)
        self.one_cycle_fit(epochs = freeze_epoch, max_lr = max_lr, wd = wd, div_factor = div_factor, pct_start = pct_start, final_div_factor = final_div_factor, **kwargs)
        
        self.unfreeze()
        
        self.one_cycle_fit(epochs = epochs, max_lr = max_lr / lr_div, wd = wd, div_factor = div_factor, pct_start = pct_start, final_div_factor = final_div_factor, **kwargs)
        
    def one_cycle_fit(self, epochs, max_lr, wd, div_factor = 15.0, pct_start= 0.25,final_div_factor =1e4, **kwargs):
        #div_factor and pct changed from default
    
        self._init_optimizer(epochs, max_lr, wd, **kwargs)
        
        
        
        self.lr_schedular = torch.optim.lr_scheduler.OneCycleLR(self.opt, max_lr = max_lr, epochs = epochs,
                                                                steps_per_epoch = len(self.dls.train), div_factor= div_factor,
                                                           pct_start = pct_start, final_div_factor = final_div_factor)
        self('before_fit')
        try:
            
            for self.epoch in self.mb:
                self('before_epoch')
                self.do_train_epoch()
                self.do_valid_epoch()
                self('after_epoch')
  
        except FitCancelException: pass
        
       
        
        self('after_fit')
    
    def _init_optimizer(self, epochs, lr, wd,params = None **kwargs):
        self.epochs = epochs
        self.lr = lr
        self.wd = wd
        
        if params is None:
            params = filter(lambda p: p.requires_grad)
            
        #eps is changed to 1e-05 from default value of 1e-08
        self.opt = self.opt_func(params = params, self.model.parameters()), lr = self.lr,
                                weight_decay = self.wd, **kwargs)
    
    def freeze_to(self, freeze_to_till, show_log = True):
        '''
            change require grad to false i.e freeze the layers
        '''
        if freeze_to_till < self.layer_groups or freeze_to_till > -self.layer_groups:

            freeze_to_till = freeze_to_till if freeze_to_till >= 0 else (self.layer_groups) + freeze_to_till

            for idx, layer in enumerate(self.model.children()):
                if idx >= freeze_to_till:
                    if show_log:
                        print(f'Layer :{idx} unFreeze')
                    for param in layer.parameters():
                        param.requires_grad = True

                else:
                    if show_log:
                        print(f'Layer :{idx} Freeze')
                    for param in layer.parameters():
                        param.requires_grad = False
                        
    def freeze(self):
        '''
            Freeze last group of layers (in this case classifier)
        '''
        self.freeze_to(freeze_to_till=-1)
        
    def unfreeze(self):
        self.freeze_to(freeze_to_till=0, show_log = False)
        
    def __call__(self, name):
        for cb in self.cbs:
            getattr(cb,name,noop)()
    
    def save_model(self, path):
        '''
        save the model
        '''
        torch.save(self.model.state_dict(), path)
   
    def load_model(self, path):
        '''
        load the model wieghts
        '''
        self.model.load_state_dict(torch.load(path))
        
    def predict(self, image, classes:list = None):
        '''args:
                input:
                    image(PIL or numpy array)
                    return the model prediction class
                    
        '''
        if isinstance(image,Image.Image):
            image_data = np.array(image.convert('RGB'))
        if isinstance(image, np.ndarray):
            assert len(image.shape) == 3, 'Image is does not have 3 channels'
            image_data = image
            
        transform = A.Compose([A.Resize(224,224),A.Normalize(), ToTensorV2()])
        image_tensor = transform(image = image_data)['image'].unsqueeze(dim = 0)
        image_tensor = image_tensor.to(self.device)
        self.model.to(self.device)
        self.model.eval()
        preds = self.model(image_tensor).cpu().detach()
        
        if hasattr(self.dls.valid.dataset,'CLASSES'):
            labels = list(self.dls.valid.dataset.CLASSES.key())
        elif classes is not None:
            labels = classes
        
        
        return {'Class':labels[preds.argmax(dim = -1).item()], 'Propability': preds}
        
        
    def get_preds(self, dl):
        '''input: 
            dataloader
            for each batch in dataloader
                get prediction from the model store it.
            
            after going through whole data from dl
            return prediction
        '''
        preds = []
        target = []
        
        for xb,yb in dl:
            
            xb = xb.to(self.device, non_blocking = True)
            yb = yb.to(self.device, non_blocking = True)
            self.model.to(self.device)
            self.model.eval()
            pred = self.model(xb).cpu().detach()
            
            preds.append(pred.argmax(dim = -1))
            if yb is not None:
                target.append(yb.cpu().detach())
            
        return torch.cat(preds),torch.cat(target)
        

SyntaxError: positional argument follows keyword argument (<ipython-input-1-e12f1ea2bffc>, line 221)

In [ ]:
#export
class ExponentialLR(_LRScheduler):
    ''' 
        ExponentialLR for LR range finder
    '''
    def __init__(self, opt, final_lr,num_iter, last_epoch = -1):
        '''
            ExponentialLR
            
            Args:
                opt (optimizer) : optimizer to use
                final_lr(float) : max lr bound
                num_iter(int) : number of iterations
                last_epoch (int): -1
        '''
        
        self.final_lr = final_lr
        self.num_iter = num_iter
        assert num_iter > 1, 'number of iteration must be grater than 1'
        
        super(ExponentialLR, self).__init__(opt, last_epoch=last_epoch)
    
    def get_lr(self):
        r = self.last_epoch / (self.num_iter - 1)
        
        return [base_lr * ((self.final_lr / base_lr)**r) for base_lr in self.base_lrs]

In [ ]:
#export
def random_seed(seed):
    '''Sets the seed value for reproducibility'''
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
#export
def seed_worker(worker_id):
    '''
        for dataloader reproducibility and avoiding duplicates
    '''
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [ ]:
#export
class FitCancelException(Exception):
    pass

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_utils.data.dataset.ipynb.
Converted 01_utils.data.dataloaders.ipynb.
Converted 02_utils.vizualize.ipynb.
Converted 03_callbacks.ipynb.
Converted 03a_hooks.ipynb.
Converted 04_learner.ipynb.
Converted 05_metrics.ipynb.
Converted 06_NIA.WOA.ipynb.
Converted index.ipynb.
